In [28]:
# https://www.kaggle.com/grouplens/movielens-20m-dataset -- dataset link

import pandas as pd
import numpy as np

In [29]:
g_tags=pd.read_csv('genome_tags.csv')
g_scores=pd.read_csv('genome_scores.csv')
movies=pd.read_csv('movie.csv')
ratings=pd.read_csv('rating.csv')
tags=pd.read_csv('tag.csv')

In [30]:
movies.shape

(27278, 3)

In [31]:
ratings.shape

(20000263, 4)

In [32]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [33]:
x=ratings['userId'].value_counts()>500
movie_lover=x[x].index.tolist()
ratings=ratings[ratings['userId'].isin(movie_lover)]
ratings.shape

(6554416, 4)

In [34]:
df=pd.merge(movies,ratings,on='movieId')

In [35]:
df.reset_index(drop=True,inplace=True)

In [36]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0,2001-07-04 07:02:29
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0,2000-11-21 21:00:21
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0,2006-04-03 10:00:08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0,2005-03-22 22:46:02


In [37]:
df.shape

(6554416, 6)

In [40]:
df_copy=df.copy()

In [41]:
movie_ratingCount = (df_copy.groupby(by = ['title'])['rating'].count().reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})[['title', 'totalRatingCount']])
movie_ratingCount.head()

,title,totalRatingCount
0,#chicagoGirl: The Social Network Takes on a Di...,2
1,$ (Dollars) (1971),7
2,$5 a Day (2008),10
3,$9.99 (2008),17
4,$ellebrity (Sellebrity) (2012),1


In [43]:
rating_with_totalRatingCount = df_copy.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,movieId,title,genres,userId,rating,timestamp,totalRatingCount
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41,6303
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0,2001-07-04 07:02:29,6303
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0,2000-11-21 21:00:21,6303
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0,2006-04-03 10:00:08,6303
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0,2005-03-22 22:46:02,6303


In [45]:
rating_with_totalRatingCount.drop(columns='timestamp',inplace=True)

In [49]:
rating_with_totalRatingCount.shape

(6554416, 6)

In [46]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   26034.000
mean      251.764
std       698.078
min         1.000
25%         3.000
50%        15.000
75%       132.000
max      6938.000
Name: totalRatingCount, dtype: float64


In [50]:
popularity_threshold = 150
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,movieId,title,genres,userId,rating,totalRatingCount
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.500,6303
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.000,6303
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.000,6303
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.000,6303
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.000,6303


In [51]:
rating_popular_movie.shape

(6108541, 6)

In [52]:
## First lets create a Pivot matrix

movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"'burbs, The (1989)",0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,2.000,0.000,4.000
'night Mother (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
(500) Days of Summer (2009),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500,0.000
*batteries not included (1987),5.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000


In [53]:
from scipy.sparse import csr_matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)


from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [61]:
movie_features_df.head()

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"'burbs, The (1989)",0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,2.000,0.000,4.000
'night Mother (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
(500) Days of Summer (2009),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500,0.000
*batteries not included (1987),5.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000


In [59]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)


distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

4328


In [126]:
movie_features_df.index[4328]

'Poltergeist III (1988)'

In [168]:
x=np.where(movie_features_df.index=='Poltergeist III (1988)')
x[0][0]

4328

In [109]:
indices

array([4328, 4327, 3954, 3952, 3953, 3951], dtype=int64)

In [122]:
indices.flatten()[1]

4327

In [119]:
print(movie_features_df.index[query_index])
print("The recommended movies are :")
for i in range(6):
    
    print(movie_features_df.index[indices.flatten()[i]])

Poltergeist III (1988)
The recommended movies are :
Poltergeist III (1988)
Poltergeist II: The Other Side (1986)
Nightmare on Elm Street 5: The Dream Child, A (1989)
Nightmare on Elm Street 3: Dream Warriors, A (1987)
Nightmare on Elm Street 4: The Dream Master, A (1988)
Nightmare on Elm Street 2: Freddy's Revenge, A (1985)


In [173]:
movies['title'].sample()

5674    Neighbors (1981)
Name: title, dtype: object

In [176]:
def movie_rec(text):
    x=np.where(movie_features_df.index==text)
    query_index=x[0][0]
    distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    print(movie_features_df.index[query_index])
    print("The recommended movies are :")
    for i in range(6):    
        print(movie_features_df.index[indices.flatten()[i]])
        
text=input("Enter a movie name :")
movie_rec(text)

Enter a movie name :Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode IV - A New Hope (1977)
The recommended movies are :
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Back to the Future (1985)
Matrix, The (1999)
